# Enter State Farm

In [2]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [3]:
%matplotlib inline
from __future__ import print_function, division
path = "/opt/notebooks/data/state-farm-distracted-driver-detection/preprocessed/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [4]:
batch_size=32

## Setup batches

In [6]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 20424 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [7]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 20424 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


Rather than using batches, we could just import all the data into an array to save some processing time. (In most examples I'm using the batches, however - just because that's how I happened to start out.)

In [ ]:
trn = get_data(path+'train')
val = get_data(path+'valid')

In [ ]:
save_array(path+'results/val.dat', val)
save_array(path+'results/trn.dat', trn)

In [ ]:
val = load_array(path+'results/val.dat')
trn = load_array(path+'results/trn.dat')

## Re-run sample experiments on full dataset

We should find that everything that worked on the sample (see statefarm-sample.ipynb), works on the full dataset too. Only better! Because now we have more data. So let's see how they go - the models in this section are exact copies of the sample notebook models.

### Single conv layer

In [8]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [9]:
model = conv1(batches)

Epoch 1/2
20424/20424 [==============================] - 380s - loss: 0.2380 - acc: 0.9421 - val_loss: 0.0431 - val_acc: 0.9920
Epoch 2/2
20424/20424 [==============================] - 202s - loss: 0.0172 - acc: 0.9982 - val_loss: 0.0278 - val_acc: 0.9950
Epoch 1/4
20424/20424 [==============================] - 205s - loss: 0.0077 - acc: 0.9993 - val_loss: 0.0187 - val_acc: 0.9975
Epoch 2/4
20424/20424 [==============================] - 213s - loss: 0.0106 - acc: 0.9982 - val_loss: 0.0203 - val_acc: 0.9960
Epoch 3/4
20424/20424 [==============================] - 204s - loss: 0.0071 - acc: 0.9989 - val_loss: 0.0119 - val_acc: 0.9975
Epoch 4/4
20424/20424 [==============================] - 202s - loss: 0.0049 - acc: 0.9995 - val_loss: 0.0464 - val_acc: 0.9855


Interestingly, with no regularization or augmentation we're getting some reasonable results from our simple convolutional model. So with augmentation, we hopefully will see some very good results.

### Data augmentation

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 20424 images belonging to 10 classes.


In [ ]:
model = conv1(batches)

Epoch 1/2
20424/20424 [==============================] - 215s - loss: 1.2435 - acc: 0.5966 - val_loss: 0.4883 - val_acc: 0.8650
Epoch 2/2
20424/20424 [==============================] - 212s - loss: 0.6287 - acc: 0.8023 - val_loss: 0.1962 - val_acc: 0.9540
Epoch 1/4
20424/20424 [==============================] - 215s - loss: 0.4411 - acc: 0.8667 - val_loss: 0.2367 - val_acc: 0.9255
Epoch 2/4
20424/20424 [==============================] - 212s - loss: 0.3418 - acc: 0.8994 - val_loss: 0.1401 - val_acc: 0.9675
Epoch 3/4
20424/20424 [==============================] - 212s - loss: 0.2828 - acc: 0.9202 - val_loss: 0.0964 - val_acc: 0.9740
Epoch 4/4
13600/20424 [==================>...........] - ETA: 64s - loss: 0.2509 - acc: 0.9276

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=15, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

20424/20424 [==============================] - 215s - loss: 0.2113 - acc: 0.9412 - val_loss: 0.0791 - val_acc: 0.9805
Epoch 2/15
 8576/20424 [===========>..................] - ETA: 110s - loss: 0.2023 - acc: 0.9410

I'm shocked by *how* good these results are! We're regularly seeing 75-80% accuracy on the validation set, which puts us into the top third or better of the competition. With such a simple model and no dropout or semi-supervised learning, this really speaks to the power of this approach to data augmentation.

### Four conv/pooling pairs + dropout

Unfortunately, the results are still very unstable - the validation accuracy jumps from epoch to epoch. Perhaps a deeper model with some dropout would help.

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [ ]:
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

In [ ]:
model.optimizer.lr=0.001

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

In [ ]:
model.optimizer.lr=0.00001

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

This is looking quite a bit better - the accuracy is similar, but the stability is higher. There's still some way to go however...

### Imagenet conv features

Since we have so little data, and it is similar to imagenet images (full color photos), using pre-trained VGG weights is likely to be helpful - in fact it seems likely that we won't need to fine-tune the convolutional layer weights much, if at all. So we can pre-compute the output of the last convolutional layer, as we did in lesson 3 when we experimented with dropout. (However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.)

In [ ]:
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [ ]:
conv_model = Sequential(conv_layers)

In [ ]:
# batches shuffle must be set to False when pre-computing features
batches = get_batches(path+'train', batch_size=batch_size, shuffle=False)

In [ ]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

In [ ]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [ ]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

### Batchnorm dense layers on pretrained conv layers

Since we've pre-computed the output of the last convolutional layer, we need to create a network that takes that as input, and predicts our 10 classes. Let's try using a simplified version of VGG's dense layers.

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [ ]:
p=0.8

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.01

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/conv8.h5')

Looking good! Let's try pre-computing 5 epochs worth of augmented data, so we can experiment with combining dropout and augmentation on the pre-trained model.

### Pre-computed data augmentation + dropout

We'll use our usual data augmentation parameters:

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

We use those to create a dataset of convolutional features 5x bigger than the training set.

In [ ]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*5)

In [ ]:
save_array(path+'results/da_conv_feat2.dat', da_conv_feat)

In [ ]:
da_conv_feat = load_array(path+'results/da_conv_feat2.dat')

Let's include the real training data as well in its non-augmented form.

In [ ]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

Since we've now got a dataset 6x bigger than before, we'll need to copy our labels 6 times too.

In [ ]:
da_trn_labels = np.concatenate([trn_labels]*6)

Based on some experiments the previous model works well, with bigger dense layers.

In [ ]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [ ]:
p=0.8

In [ ]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Now we can train the model as usual, with pre-computed augmented data.

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.01

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.0001

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Looks good - let's save those weights.

In [ ]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

### Pseudo labeling

We're going to try using a combination of [pseudo labeling](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf) and [knowledge distillation](https://arxiv.org/abs/1503.02531) to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. At a later date we'll try using the test set.

To do this, we simply calculate the predictions of our model...

In [ ]:
val_pseudo = bn_model.predict(conv_val_feat, batch_size=batch_size)

...concatenate them with our training labels...

In [ ]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])

In [ ]:
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

...and fine-tune our model using that data.

In [ ]:
bn_model.load_weights(path+'models/da_conv8_1.h5')

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.00001

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

That's a distinct improvement - even although the validation set isn't very big. This looks encouraging for when we try this on the test set.

In [ ]:
bn_model.save_weights(path+'models/bn-ps8.h5')

### Submit

We'll find a good clipping amount using the validation set, prior to submitting.

In [ ]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [ ]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval()

In [ ]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [ ]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [ ]:
subm = do_clip(preds,0.93)

In [ ]:
subm_name = path+'results/subm.gz'

In [ ]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [ ]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[4:] for a in test_filenames])
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [ ]:
FileLink(subm_name)

This gets 0.534 on the leaderboard.

## The "things that didn't really work" section

You can safely ignore everything from here on, because they didn't really help.

### Finetune some conv layers too

In [ ]:
for l in get_bn_layers(p): conv_model.add(l)

In [ ]:
for l1,l2 in zip(bn_model.layers, conv_model.layers[last_conv_idx+1:]):
    l2.set_weights(l1.get_weights())

In [ ]:
for l in conv_model.layers: l.trainable =False

In [ ]:
for l in conv_model.layers[last_conv_idx+1:]: l.trainable =True

In [ ]:
comb = np.concatenate([trn, val])

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=8, height_shift_range=0.04, 
                shear_range=0.03, channel_shift_range=10, width_shift_range=0.08)

In [ ]:
batches = gen_t.flow(comb, comb_pseudo, batch_size=batch_size)

In [ ]:
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

In [ ]:
conv_model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
conv_model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, 
                 nb_val_samples=val_batches.N)

In [ ]:
conv_model.optimizer.lr = 0.0001

In [ ]:
conv_model.fit_generator(batches, batches.N, nb_epoch=3, validation_data=val_batches, 
                 nb_val_samples=val_batches.N)

In [ ]:
for l in conv_model.layers[16:]: l.trainable =True

In [ ]:
conv_model.optimizer.lr = 0.00001

In [ ]:
conv_model.fit_generator(batches, batches.N, nb_epoch=8, validation_data=val_batches, 
                 nb_val_samples=val_batches.N)

In [ ]:
conv_model.save_weights(path+'models/conv8_ps.h5')

In [ ]:
conv_model.load_weights(path+'models/conv8_da.h5')

In [ ]:
val_pseudo = conv_model.predict(val, batch_size=batch_size*2)

In [ ]:
save_array(path+'models/pseudo8_da.dat', val_pseudo)

### Ensembling

In [ ]:
drivers_ds = pd.read_csv(path+'driver_imgs_list.csv')
drivers_ds.head()

In [ ]:
img2driver = drivers_ds.set_index('img')['subject'].to_dict()

In [ ]:
driver2imgs = {k: g["img"].tolist() 
               for k,g in drivers_ds[['subject', 'img']].groupby("subject")}

In [ ]:
def get_idx(driver_list):
    return [i for i,f in enumerate(filenames) if img2driver[f[3:]] in driver_list]

In [ ]:
drivers = driver2imgs.keys()

In [ ]:
rnd_drivers = np.random.permutation(drivers)

In [ ]:
ds1 = rnd_drivers[:len(rnd_drivers)//2]
ds2 = rnd_drivers[len(rnd_drivers)//2:]

In [ ]:
models=[fit_conv([d]) for d in drivers]
models=[m for m in models if m is not None]

In [ ]:
all_preds = np.stack([m.predict(conv_test_feat, batch_size=128) for m in models])
avg_preds = all_preds.mean(axis=0)
avg_preds = avg_preds/np.expand_dims(avg_preds.sum(axis=1), 1)

In [ ]:
keras.metrics.categorical_crossentropy(val_labels, np.clip(avg_val_preds,0.01,0.99)).eval()

In [ ]:
keras.metrics.categorical_accuracy(val_labels, np.clip(avg_val_preds,0.01,0.99)).eval()